In [2]:
import cv2
import os
import numpy as np
import glob
import pandas as pd
import re

In [3]:
def read_names(methods):

    for method in methods:
        folder_a =  'faceforensics_data/forensics_split/' + method + '/authentic'
        folder_f =  'faceforensics_data/forensics_split/' + method + '/fake'
        
        images_a = glob.glob(folder_a + "/*.png")
        images_f = glob.glob(folder_f + "/*.png")
        

    return images_a + images_f

images = read_names(['validation', 'train'])

In [4]:
images = [el.replace('train/', '').replace('validation/', '')
          .replace('faceforensics_data/forensics_split/','') for el in images]

print(len(images))
print(images[0])

113928
authentic/Original_749_0675.png


In [5]:
def build_dataframe(filenames):
    index = range(len(filenames))
    df = pd.DataFrame(index = index, columns = ['method', 'video', 'image', 'dummy'])
    df = df.fillna(0)
    methods = [el[el.find('/')+1: el.find('_')] for el in filenames]
    video_numbers = [re.search("_(.*?)\_", el).group(1) for el in filenames]
    image_numbers = [el[el.find('_')+1: el.find('.')][4:] for el in filenames]
    true_labels = [0 if el[0] == 'a' else 1 for el in filenames]
    
    df['method'] = methods
    df['video'] =  video_numbers
    df['image'] =  image_numbers
    df['dummy'] = [1] * len(filenames)
    
    return df

In [6]:
data = build_dataframe(images)
display(data[10:13])

,method,video,image,dummy
10,Original,508,0525,1
11,Original,168,0615,1
12,Original,604,0405,1


In [7]:
df = data.groupby(['method', 'video'])\
                    ['dummy'].apply(list).reset_index()
    
collected_labels_pred = list(df['dummy'])
num_per_video = [len(el) for el in collected_labels_pred]
    
df['num frames'] = num_per_video
display(df[0:3])

,method,video,dummy,num frames
0,Deepfakes,001,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",31
1,Deepfakes,002,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",47
2,Deepfakes,005,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",26


In [8]:
print(np.min(num_per_video))
print(np.max(num_per_video))
print(np.mean(num_per_video))

15
100
31.64666666666667
